In [ ]:
import random
import math
import numpy as np
import gmpy2
from sympy import * 

def factor_(B,A,sol,num):
    Z = 1
    X2 = 1
    #print sol
    for i in range(len(sol)):
        if sol[i]==1:
            #print is_square( int(mod(B[i],num)))
            Z = Z*B[i]
            X2 = X2*A[i]**2
            # Z^2 = X2 mod n
    return gcd(sqrt(Z)-sqrt(X2),num)

def pivot(v):
    # v is a binary list
    # output the entry of the first 1
    for i in range(len(v)):
        if v[i]==1:
            return i
        
def take_the_free_unknowns(C):
    # all the elements under the pivot of last element of C corresponds to free unknowns
    lst = []
    lenC = len(C)
    #l = matrix(C).dimensions()[1]
    num_rows, num_cols = np.array(C).shape
    l=num_cols
    piv = pivot(C[lenC-1])
    #L = matrix( (matrix(C).transpose()[piv+1:l]).transpose() )
    temp = piv + 1
    L = np.array((np.array(C).transpose()[temp:l]).transpose())
    lst = [i for i in range(piv+1,l)]       
    for i in range(lenC-2,-1,-1):
        a = pivot(C[i+1])-pivot(C[i])
        if a>1:
            lst1 = []
            lst1 =  [j for j in range(pivot(C[i])+1,pivot(C[i])+a)]
            lst = lst + lst1
            # ERROOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOR
            #R1 = matrix( (matrix(C).transpose()[pivot(C[i])+1:pivot(C[i])+a]).transpose() )
            R1 = np.array( (np.array(C).transpose()[pivot(C[i])+1:pivot(C[i])+a]).transpose() )
            #L = block_matrix([[R1,L]])
            L = np.block([[R1,L]])
    lst = flatten(lst)[::-1]

    num_rows, num_cols = L.shape
    
    # return L,L.dimensions()[1],sorted(flatten(lst))
    return L,num_cols,sorted(flatten(lst))


def pick_a_random_solution(T):
    C = T[0]
    R = take_the_free_unknowns(C)
    print("pick a random solution")
    #v = random_matrix(GF(2), 1, R[1]).transpose()
    v= (np.random.randint(2,size=(1,R[1]))).transpose()
    #basic_unknowns = (R[0])*v
    basic_unknowns = np.matmul(R[0],v)
    lst = [-1 for i in range(len(C[0]))]
    
    j = 0
    for i in R[2]:
        lst[i] = v[j][0]
        j = j + 1
    j = 0
    for i in range(len(C[0])): 
        if int(lst[i])==-1:
            lst[i] = basic_unknowns[j][0]
            j = j + 1
    if T[1]==-1000:
        lst = lst + [0]
    return lst

def take_cols(M):   
    # it takes a matrix M and return a list with the non zero columns of M
    #dim = M.dimensions()
    num_rows, num_cols = M.shape
    dim=(num_rows,num_cols)
    col = []
    col.append(  [ M[0:dim[0],i] for i in range(dim[1]) ] )
    col = col[0]

    #col = [col[i].list() for i in  range(len(col))]
    #print("col after print ", col)
    C = []
    for i in range(len(col)):
        # edo elexei an apla den exei mono midenika???????????
        #if col[i]!=dim[0]*[0]:
            
        if not(all(v == 0 for v in col[i])):
           C.append(col[i])
    return C

def are_the_same(list1, list2):
    if len(list1)!=len(list2):
        return False
    else:
        for i in range(len(list1)):
            if list1[i]!=list2[i]:
                return False
        return True 
    
    
def initialization_for_C(M):
    C = take_cols(M)
    
    temp1 = (len(C[0])-1)*[0]+[1]
    temp2 = C[len(C)-1]       
    if are_the_same(temp1,temp2):
        C.pop(len(C)-1)
        return C,1
    else:
        return C,-1000

    
def trial_division(n):
    counter   = 0
    counter_2 = 0
    a = []
    if (n<0): #if n is negative add -1 to list
        a.append(-1)
        n=n*(-1)
    while n % 2 == 0:
        a.append(2)
        n /= 2 # set as n the integer n/2
        counter = counter + 1
        counter_2 = counter_2 + counter
    f = 3 # f is a possible prime divisor. We shall search up to sqrt(n).
    while f^2 <= n:
        if n % f == 0: # similar as previous. We check if f divides n.
            a.append(f) 
            n /= f    # we update n with n/f
            counter = counter + 1
            counter_2 = counter_2 + counter
        else:
            f += 2   # we increase the possible divisor f by 2. So f = 3,5,7,9,11....
            counter_2 = counter_2 +  1 
    if n != 1: a.append(gmpy2.mpz(n)) # put n in the list of divisors of n
    # Only odd number is possible
    #print "counter, all loops:",counter,counter_2
    #print product(a)
    return a

def random_prime(bits):
    while 1==1:
        N = random.getrandbits(bits)
        if millerRabin(N, 8):
            break
    return N

def rsa_modulus(bits):
    #creating random prime numbers
    p = random_prime(int(np.floor(bits/2)))
    q = random_prime(int(np.floor(bits/2)))
    #checking if p and q is equal, until I create different prime numbers
    if p!=q:
        return p,q,p*q


def rank_of_echelon_form_matrix(B):
    zero_list = [0]*B.dimensions()[1]
    num_rows = B.dimensions()[0]
    upper = 0
    lower = num_rows
    while(abs(lower-upper)!=1):
        mean = floor((upper+lower)/2)
        if B[mean] != vector(zero_list):
            upper=  mean
        else:
            if B[mean+1] == vector(zero_list):
                lower = mean
    return max(lower,upper)
    
def linear_indepedent_rows(p,B): 
    vec_space =  VectorSpace(GF(p),B.dimensions()[1])
    subspace =  vec_space.subspace([ vector(B[i]) for i in range(B.dimensions()[0]) ])
    dim = subspace.dimension()
    print ('lines : ', B.dimensions()[0])
    print ('indepedent lines :',dim)
    print ('dimension of the matrix',B.dimensions())
    print ('difference (dim null):',B.dimensions()[0]-dim)
    return 
    

def bits(x):
    ''' returns the number of bits of the positive integer x'''
    if x>0:
        return floor(log(x,2))+1;
    else:
        return 0

def decimalToBinary(n):
    return bin(n).replace("0b", "")

def string_to_list_of_ints(l):
    l = list(l)
    for i in range(len(l)):
        l[i] = int(l[i])  # make elements int
    return l

def fast(base, power, modulo):
    g = decimalToBinary(power)
    g = string_to_list_of_ints(g)
    x = base
    d = 1
    for i in range(len(g) - 1, -1, -1):
        if g[i] == 1:
            d = d * x % modulo
        x = (x * x) % modulo
    return d

def millerRabin(n, k):
    i = 1
    j = 0
    if n & 1 == 0:    #print C
    #if R=="todo":
    #    return "todo"
        return False
    else:
        """
        write n-1 as 2^s * t found from
        https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Primality_Testing
        fbclid=IwAR3AENaDRikz49KC5TiZngp4nOda19z3c8tqpisZop7A1Ju0ZdIoS7WoZrE
        """
        s, t = 0, n - 1
        while t & 1 == 0:
            s, t = s + 1, t >> 1
        while i <= k:
            a = random.randint(2, n - 1)
            b = fast(a, t, n)
            if b % n == 1:
                j = j + 1
                if j == k:
                    return True
            for r in range(s):
                if b % n == n - 1:
                    j = j + 1
                    if j == k:
                        return True
                b = fast(b, 2, n)
            i = i + 1
        return False


def isPrime(n):
    lowPrimes = Primes(100)[0:1000]
    if n >= 3:
        if n & 1 != 0:
            if n in lowPrimes:
                return True
            return millerRabin(n, 40)
    return False
    
#Sieve of Eratosthenes https://www.geeksforgeeks.org/python-program-for-sieve-of-eratosthenes/
def SieveOfEratosthenes(n):
    prime = [True for i in range(int(n) + 1)] #maybe it should be +2 
    p = 2
    while (p * p <= n): 
        # If prime[p] is not changed, then it is a prime 
        if (prime[p] == True): 
              
            # Update all multiples of p 
            for i in range(p * 2, int(n) + 1, p): 
                prime[i] = False
        p += 1
    prime[0]= False
    prime[1]= False
    # Print all prime numbers 
    primes_list = []
    for p in range(int(n) + 1): 
        if prime[p]: 
            primes_list.append(p) 
    return primes_list
    
#from sage.rings.factorint import factor_trial_division
#import sympy

def is_smooth(b,S):
    b_fact=[i for i in trial_division(b)]
    for i in range(len(b_fact)):
        if not b_fact[i] in S:
            return False
    return True


def Ln(x):
    return math.exp(math.sqrt(math.log(x)*(math.log(math.log(x)))))

def legendre_symbol(n, k):
    #https://en.wikipedia.org/wiki/Jacobi_symbol
    assert(k > 0 and k % 2 == 1)
    n = n % k
    t = 1
    while n != 0:
        while n % 2 == 0:
            n = n / 2
            r = k % 8
            if (r == 3 or r == 5):
                t = -t
        n, k = k, n
        if (n % 4 == 3 and k % 4 == 3):
            t = -t
        n = n % k
    if k == 1:
        return t
    else:
        return 0

def exponent(set, base):
    # returns how many times "base" exists in "set"
    count = 0 
    for i in range(len(set)):
        if set[i]==base:
            count+=1
    return count
   
def row_echelon(A): #ΙΣΩΣ ΝΑ ΜΗΝ ΜΑΣ ΚΑΝΕΙ!!!!!!!!!!!!!!!!
    """
    https://math.stackexchange.com/questions/3073083/how-to-reduce-matrix-into-row-echelon-form-in-python
    """
    """ Return Row Echelon Form of matrix A """

    # if matrix A has no columns or rows,
    # it is already in REF, so we return itself
    r, c = A.shape
    if r == 0 or c == 0:
        return A

    # we search for non-zero element in the first column
    for i in range(len(A)):
        if A[i,0] != 0:
            break
    else:
        # if all elements in the first column is zero,
        # we perform REF on matrix from second column
        B = row_echelon(A[:,1:])
        # and then add the first zero-column back
        return np.hstack([A[:,:1], B])

    # if non-zero element happens not in the first row,
    # we switch rows
    if i > 0:
        ith_row = A[i].copy()
        A[i] = A[0]
        A[0] = ith_row

    # we divide first row by first element in it
    A[0] = A[0] / A[0,0]
    # we subtract all subsequent rows with first row (it has 1 now as first element)
    # multiplied by the corresponding element in the first column
    A[1:] -= A[0] * A[1:,0:1]

    # we perform REF on matrix from second row, from second column
    B = row_echelon(A[1:,1:])

    # we add first row and first (zero) column, and return
    return np.vstack([A[:1], np.hstack([A[1:,:1], B]) ])

def quadratic_sieve(n,B,parameter):
    primes = []
    primes = SieveOfEratosthenes(B) 
    #print primes
    
    # The Factor Base: prime numbers that have a Legendre symbol equal to 1
    factor_base=[-1,2]
    for i in range(1,len(primes)):
        if legendre_symbol(n,primes[i])==1:                                
            factor_base.append(primes[i])          
    
    print ("Factor Base:",factor_base)                                  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Print
    m = math.floor(math.sqrt(n))
    print ("floor(sqrt(n)), m:",m)                                      #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Print
    t=len(factor_base)  # the number of columns
    print ("length of factor base, t:",t)                                #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Print  
    
    if parameter == 1:
        parameter = t+2
    num_of_rows = parameter    # default = t + 2
    num_of_cols = t
    i = 1
    x = 0                                          #the x values are chosen in the order 0, ±1, ±2,..
    change_x=1
    a=[]                                           #the list with B-smooth values x[i] of the form \pm (x+m) for some x in [0,1,...,t+1]
    #u = matrix(GF(2), num_of_rows, num_of_cols)
    u = np.zeros( (num_of_rows, num_of_cols) )
    #e = matrix(GF(2), num_of_rows, num_of_cols)
    e = np.zeros( (num_of_rows, num_of_cols) )
    bs=[]    
    #print ("\n","smooth numbers <B with factors in factor base F:","\n")  #the list of B-smooth c[i] = x[i]^2 - n
    #print ("we need ",t+2," relations","\n")
    print ("we shall compute ",parameter," relations")
    while i<=num_of_rows: #since i starts from 1 
        b = ((x+m)**2)- n   #test using trial division by elements in factor_base whether b is pt-smooth
        #print("b:",b," x:",x," m",m,"n:",n)
        if is_smooth(b,factor_base):  
            #ebgala to and is_square(b)!=True apo ton elegxo
            a.append(x+m) 
            bs.append(b)
            b_set = [j for j in trial_division(b)]
            if i%5==0:
                print (i,":", b_set,":", b)
            
            if b<0: 
                #if q(xi)<0 then the first factor in factor base is 1 representing -1
                #ui[0]=1
                u[i-1,0]=1 # i-1 because i starts from 1 
                e[i-1,0]=1
            for j in range(len(b_set)):
                #ui[factor_base.index(b_set[j][0])]= int(mod(b_set[j][1],2))
                #b_set[j][0] einai o prime factor kai me to index brisko tin thesi tou, diladi ti stili                     
                #sto u pinaka
                u[i-1,factor_base.index(b_set[j])]= exponent(b_set,b_set[j])%2
                e[i-1,factor_base.index(b_set[j])]= exponent(b_set,b_set[j])           
            i = i+1   
        #pick new x
        if change_x==1:
            if x<0:
                x=x*(-1)
            x=x+1
        else:
            x=x*(-1)
        change_x=change_x * (-1)
        
    #print (rank_of_echelon_form_matrix(u.echelon_form()))
    #print (linear_indepedent_rows(2,u))                                 
                                    
    #eche = (u.transpose().echelon_form()).transpose() 
    #print("u before eche ",u)
    eche = row_echelon(u.transpose())
    eche = eche.transpose() 
    
    return eche,bs,a


def main():
    #num = 24961
    num= 78996566
    #num=87463
    #num = 773978585664881 
    #num =  nth_prime(881989980980088)*nth_prime(98199090887898)
    #p,q,num = rsa_modulus(20) # generate a rsa modulus of 40 bits
    #num = 9007203214164403
    #print ("n,p,q:",num,p,q)
    #print ("bits:",bits(num))
    print("num ",num)
    B_limit=Ln(math.pow(num,0.3))   # you have to choose the suitable exponent
    print ("B:",math.floor(B_limit))
    parameter = 1  # if parameter is 1 then we set parameter <-- t+2
    M,B,A = quadratic_sieve(num,B_limit,parameter)
    #print (M.dimensions())
    print("bs: ", B)
    print("a: ",A)
    #print("main M ", M )
    

    C = initialization_for_C(M);
    print("Psaxno liseis")
    for i in range(2**7): # TODO
        sol = pick_a_random_solution(C)
        #print ("sol ",sol)
        F = factor_(B,A,sol,num) 
        
        if F>1 and F<num:
            f2 = num / F
            print("Factors: ",F,f2)
            print("brika lisi OMG OMG OMG")
            break



if __name__ == "__main__":
    main()


num  78996566
B: 20
Factor Base: [-1, 2, 5, 13]
floor(sqrt(n)), m: 8888
length of factor base, t: 4
we shall compute  6  relations


### Πιο κάτω έχω κώδικα για trial division. Κάτι άλλο που ίσως χρειαστεις είναι η βιβλιοθηκη gmpy2. 
##### Για να δεις τις μεθόδους που έχει αυτη η βιβλιοθηκη, πας σε ενα κελι γράφεις gmpy2. και πατάς tab.

In [4]:
import gmpy2

def trial_division(n):
    counter   = 0
    counter_2 = 0
    a = []
    if (n<0): #if n is negative add -1 to list
        a.append(-1)
        n=n*(-1)
    while n % 2 == 0:
        a.append(2)
        n /= 2 # set as n the integer n/2
        counter = counter + 1
        counter_2 = counter_2 + counter
    f = 3 # f is a possible prime divisor. We shall search up to sqrt(n).
    while f^2 <= n:
        if n % f == 0: # similar as previous. We check if f divides n.
            a.append(f) 
            n /= f    # we update n with n/f
            counter = counter + 1
            counter_2 = counter_2 + counter
        else:
            f += 2   # we increase the possible divisor f by 2. So f = 3,5,7,9,11....
            counter_2 = counter_2 +  1 
    if n != 1: a.append(gmpy2.mpz(n)) # put n in the list of divisors of n
    # Only odd number is possible
    #print "counter, all loops:",counter,counter_2
    #print product(a)
    return a

a = trial_division(-25060)
print (a)
print(a[1])

[-1, 2, 2, 5, 7, 179]
2


[3, mpz(17)]

In [ ]:
import numpy as np


def row_echelon(A):
    """
    https://math.stackexchange.com/questions/3073083/how-to-reduce-matrix-into-row-echelon-form-in-python
    """
    """ Return Row Echelon Form of matrix A """

    # if matrix A has no columns or rows,
    # it is already in REF, so we return itself
    r, c = A.shape
    if r == 0 or c == 0:
        return A

    # we search for non-zero element in the first column
    for i in range(len(A)):
        if A[i,0] != 0:
            break
    else:
        # if all elements in the first column is zero,
        # we perform REF on matrix from second column
        B = row_echelon(A[:,1:])
        # and then add the first zero-column back
        return np.hstack([A[:,:1], B])

    # if non-zero element happens not in the first row,
    # we switch rows
    if i > 0:
        ith_row = A[i].copy()
        A[i] = A[0]
        A[0] = ith_row

    # we divide first row by first element in it
    A[0] = A[0] / A[0,0]
    # we subtract all subsequent rows with first row (it has 1 now as first element)
    # multiplied by the corresponding element in the first column
    A[1:] -= A[0] * A[1:,0:1]

    # we perform REF on matrix from second row, from second column
    B = row_echelon(A[1:,1:])

    # we add first row and first (zero) column, and return
    return np.vstack([A[:1], np.hstack([A[1:,:1], B]) ])

A = np.array([[4, 7, 3, 8],
              [8, 3, 8, 7],
              [2, 9, 5, 3]])

row_echelon(A)

In [ ]:
import math
import itertools
import numpy as np

def mspp(P,c,L):
    '''
    P={ui}i, gcd(ui,Λ)=1, |P|=n+1
    '''
    #1
    n = len(P) - 1 
    I1 = list(range(0,math.ceil(n/2)+1)) #range([,))
    I2 = list(range(math.ceil(n/2)+1,n+1))
    print("Is are ready")
    
    #2
    e1 = list(map(list, itertools.product([0, 1], repeat=len(I1)))) #all combination of n-binary value
    rows = pow(2,len(I1))
    print("rows of e1")
    U1 = [] 
    for i in range(0,rows):
        product = 1
        for j in range(0,len(I1)):
            product = product * pow(P[j],e1[i][j])
        U1.append(product)
    print("len U1", len(U1))
    
    #3  
    U2 = []
    if len(I1)!=len(I2):
        e2 = list(map(list, itertools.product([0, 1], repeat=len(I2)))) 
        rows = pow(2,len(I2))

    print(rows, len(e2))
    print(len(I2),len(e2[0]))
    for i in range(0,rows):
        product = c
        for j in range(0,len(I2)):
            product = product * pow(P[j],-e2[i][j])
        U2.append(product)
        
    #4
    #inter = set(U1).intersection(U2)
    intersection_indexes = np.in1d(U1, U2).nonzero()
    if any(map(len, intersection_indexes)):
        #5 Let y be an element of U1 ∩ U2
        # η γραμμή <intersection_indexes[0][0]> του πίνακα e1 
        # περιέχει τους δείκτες [0,1] που θα χρησιμοποιήσουμε στο ui
        B = []
        for y in range(len(intersection_indexes[0])):
            product = 1
            for i in range(len(I1)):
                B.append(pow(P[j],e1[intersection_indexes[0][y],i]))
                product = product * pow(P[j],e1[intersection_indexes[0][y],i])
            print("B",B)
            if product%L == c:
                # 6: return ui:Πui ≡ c mod Λ.
                return B
        return False
    else:
        #7
        return False
        
        
        
        
        
P = range(2,pow(10,7))
print("P is ready")
L= 1000001
c = 19023
mspp(P,c,L)
        

In [10]:
a = np.array(range(11))
a = a[::-1]
b = np.array([13,15])
print("test")
print(np.intersect1d(a, b))
indexes = np.in1d(a, b).nonzero()
print(type(indexes))

if any(map(len, indexes)):
    print('not empty')
else:
    print("empty")


test
[]
<class 'tuple'>
empty
